In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
import json
from pathlib import Path
from tqdm import tqdm
import pickle

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud
from nuscenes.utils.geometry_utils import view_points, transform_matrix
from pyquaternion import Quaternion
import cv2

from YOLORadarModel import ProbabilisticYOLORadar3D
from losses import ProbabilisticLoss
from utils import utils, dataLoader
from utils.utils import TrainingVisualizer

ImportError: cannot import name 'TrainingVisualizer' from 'utils.utils' (e:\Radar and RGB Camera Sensor Fusion\nuScenes Dataset\utils\utils.py)

Inputs:
├── Front Camera Image (900×1600×3) at time t
└── Front Radar Point Cloud (N×5) with timestamps [t-Δt to t]
    ↓
Probabilistic Radar Processing
├── Temporal Alignment & Uncertainty Modeling
├── 3D Gaussian Sphere Generation
└── Uncertainty-Aware Feature Encoding
    ↓
Feature Extraction (Dual Branch)
├── Image Branch: Modified YOLOv8 Backbone
└── Radar Branch: Probabilistic PointNet Encoder
    ↓
Uncertainty-Aware Fusion Module
    ↓
Modified Neck (FPN + PAN)
    ↓
3D Detection Head with Uncertainty Estimation
├── 2D Box Regression
├── 3D Box Regression (center, dimensions, rotation)
├── Depth Estimation + Uncertainty
├── Velocity Estimation + Uncertainty
└── Classification with Confidence
    ↓
Outputs: [class, 2D box, 3D center, 3D dims, yaw, velocity] 
         + [uncertainties for each prediction]

Radar Point Representation:
├── Mean (μ): (x, y, z, vx, vy) - measured values
├── Covariance (Σ): 5×5 matrix - measurement uncertainty
├── Temporal offset: Δt = t_image - t_radar
└── Radar Cross Section (RCS): measurement confidence

Gaussian Parameters per point:
├── Position uncertainty: σ_pos (from range accuracy)
├── Velocity uncertainty: σ_vel (from Doppler accuracy)  
├── Temporal uncertainty: σ_temp (from time difference)
└── RCS-based confidence: weight = f(RCS)

For each radar point at time (t - Δt):

1. Ego Motion Compensation:
   p_compensated = p_measured - ego_velocity × Δt
   
2. Temporal Uncertainty Propagation:
   σ_temporal² = σ_vel² × (Δt)²
   
3. Combined Position Uncertainty:
   Σ_pos = [
       [σ_x² + σ_temporal², 0, 0],
       [0, σ_y² + σ_temporal², 0],
       [0, 0, σ_z²]
   ]
   
4. Velocity Uncertainty (considering time lag):
   Σ_vel = [
       [σ_vx² + k×Δt, 0],
       [0, σ_vy² + k×Δt]
   ]
   where k is uncertainty growth rate

In [ ]:

def main():
    """Main training function"""
    
    # ========== Configuration ==========
    config = {
        'dataroot': '/data/sets/nuscenes',
        'version': 'v1.0-trainval',  # or 'v1.0-mini' for testing
        'batch_size': 4,
        'num_workers': 8,
        'num_epochs': 100,
        'learning_rate': 1e-4,
        'weight_decay': 1e-4,
        'image_size': (640, 640),
        'max_radar_points': 1000,
        'save_dir': './checkpoints',
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'resume': None,  # Path to checkpoint to resume from
    }
    
    print("Configuration:")
    for key, value in config.items():
        print(f"  {key}: {value}")
    
    # Create save directory
    os.makedirs(config['save_dir'], exist_ok=True)
    
    # Save config
    with open(os.path.join(config['save_dir'], 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # ========== Create Datasets ==========
    print("\nCreating datasets...")
    train_dataset = dataLoader.NuScenesRadarDataset(
        dataroot=config['dataroot'],
        version=config['version'],
        split='train',
        image_size=config['image_size'],
        max_radar_points=config['max_radar_points']
    )
    
    val_dataset = dataLoader.NuScenesRadarDataset(
        dataroot=config['dataroot'],
        version=config['version'],
        split='val',
        image_size=config['image_size'],
        max_radar_points=config['max_radar_points']
    )
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=config['num_workers'],
        collate_fn=utils.collate_fn,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=config['num_workers'],
        collate_fn=utils.collate_fn,
        pin_memory=True
    )
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Val samples: {len(val_dataset)}")
    
    # ========== Create Model ==========
    print("\nCreating model...")
    model = ProbabilisticYOLORadar3D(num_classes=10)
    model = model.to(config['device'])
    
    # Count parameters
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Model parameters: {num_params:,}")
    
    # ========== Create Optimizer and Loss ==========
    optimizer = optim.AdamW([
        {'params': model.image_backbone.parameters(), 'lr': config['learning_rate'] * 0.1},
        {'params': model.radar_branch.parameters(), 'lr': config['learning_rate']},
        {'params': model.fusion_p3.parameters(), 'lr': config['learning_rate']},
        {'params': model.fusion_p4.parameters(), 'lr': config['learning_rate']},
        {'params': model.fusion_p5.parameters(), 'lr': config['learning_rate']},
        {'params': model.head_3d.parameters(), 'lr': config['learning_rate']},
    ], weight_decay=config['weight_decay'])
    
    criterion = ProbabilisticLoss(
        loss_weights={
            'cls': 1.0,
            'box_2d': 2.0,
            'depth': 2.0,
            'center': 2.0,
            'dims': 1.0,
            'rot': 0.5,
            'vel': 1.5,
            'uncertainty_reg': 0.1
        }
    )
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=config['num_epochs'],
        eta_min=1e-6
    )
    
    # Resume from checkpoint
    start_epoch = 0
    best_val_loss = float('inf')
    
    if config['resume']:
        print(f"\nResuming from checkpoint: {config['resume']}")
        checkpoint = utils.load_checkpoint(config['resume'])
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_loss = checkpoint.get('best_val_loss', float('inf'))
    
    viz = utils.TrainingVisualizer(save_dir='./logs', experiment_name = 'test_run')
    # ========== Training Loop ==========
    print("\nStarting training...")
    
    for epoch in range(start_epoch, config['num_epochs']):
        print(f"\n{'='*60}")
        print(f"Epoch {epoch}/{config['num_epochs']-1}")
        print(f"{'='*60}")
        
        # Train
        train_loss = utils.train_one_epoch(
            model, train_loader, criterion, optimizer, config['device'], epoch
        )
        
        # Validate
        val_loss = utils.validate(model, val_loader, criterion, config['device'])
        
        # Update learning rate
        scheduler.step()
        viz.log_epoch(epoch, train_loss, val_loss, scheduler.get_last_lr())
        print(f"\nEpoch {epoch} Summary:")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Save checkpoint
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'best_val_loss': best_val_loss,
            'config': config
        }
        
        # Save last checkpoint
        utils.save_checkpoint(
            checkpoint,
            os.path.join(config['save_dir'], 'last.pth')
        )
        
        # Save best checkpoint
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            checkpoint['best_val_loss'] = best_val_loss
            utils.save_checkpoint(
                checkpoint,
                os.path.join(config['save_dir'], 'best.pth')
            )
            print(f"  New best model saved! (Val Loss: {val_loss:.4f})")
        
        # Save periodic checkpoint
        if (epoch + 1) % 10 == 0:
            utils.save_checkpoint(
                checkpoint,
                os.path.join(config['save_dir'], f'epoch_{epoch}.pth')
            )
    
    print("\n" + "="*60)
    print("Training completed!")
    print(f"Best validation loss: {best_val_loss:.4f}")
    print("="*60)
    viz.plot_summary()
    viz.generate_report()

if __name__ == '__main__':
    main()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
    
model = ProbabilisticYOLORadar3D(num_classes=10).to(device)
    
# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {num_params:,}")
    
# Test forward pass
batch_size = 2
image = torch.randn(batch_size, 3, 640, 640).to(device)
radar_points = torch.randn(batch_size, 1000, 6).to(device)
radar_timestamps = torch.zeros(batch_size, 1000).to(device)
calib = [None] * batch_size
    
print("\nRunning forward pass...")
with torch.no_grad():
    outputs = model(image, radar_points, radar_timestamps, calib)
    
print("\nOutput shapes:")
for key, value in outputs.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape}")
    elif isinstance(value, dict):
        print(f"  {key}: dict with {len(value)} items")
    
print("\nModel test passed!")

cuda
Total parameters: 74,958,695

Running forward pass...

Output shapes:
  cls: torch.Size([2, 10, 80, 80])
  box_2d: torch.Size([2, 4, 80, 80])
  depth_mean: torch.Size([2, 1, 80, 80])
  depth_std: torch.Size([2, 1, 80, 80])
  center_offset: torch.Size([2, 2, 80, 80])
  dims_mean: torch.Size([2, 3, 80, 80])
  rot_mean: torch.Size([2, 2, 80, 80])
  vel_mean: torch.Size([2, 2, 80, 80])
  vel_std: torch.Size([2, 2, 80, 80])
  fusion_weights: dict with 3 items

✓ Model test passed!


In [ ]:
 # Create visualizer
viz = TrainingVisualizer(save_dir='./logs', experiment_name='test_run')
    
# Simulate training for demo
print("Simulating training...")
for epoch in range(50):
    # Fake losses that improve over time
    train_total = 5.0 * np.exp(-epoch/20) + np.random.normal(0, 0.1)
    val_loss = 4.8 * np.exp(-epoch/20) + np.random.normal(0, 0.15)
        
    train_losses = {
        'total': train_total,
        'cls': train_total * 0.3,
        'box_2d': train_total * 0.25,
        'depth': train_total * 0.25,
        'vel': train_total * 0.2
    }
        
    lr = 1e-4 * (0.95 ** epoch)
        
    viz.log_epoch(epoch, train_losses, val_loss, lr)
    
# Generate all plots
viz.plot_summary()
viz.generate_report()
    
print("\n✓ Visualizer test completed!")

AttributeError: module 'utils.utils' has no attribute 'TrainingVisualizer'